In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/code/lawsdiv`


In [2]:
using DataFrames, DataFramesMeta, GLM
using Statistics, StatsBase
using SparseArrays
using FHist
using Plots

In [3]:
include("./Data_Input.jl")
using .DataImport

In [4]:
sep_data = DataImport.GetCrossSecData("./Data/crosssecdata.RData"; min_samples=10, min_counts=0, min_nreads=1000);

In [5]:
function make_LRM(x, portion=1)
    
    S = Int64(floor(length(x) * portion))
    lx = log.(x[1:S])
    M = repeat(lx, 1, S)
    LRM = M - M'
    LRM[isnan.(LRM)] .= 0.0
    LRM[isinf.(LRM)] .= 0.0
    
    return LRM
end

make_LRM (generic function with 2 methods)

In [ ]:
fig = plot()
env_matrix = Dict()
for (key, data) in sep_data
    println(key)

    # Compute frequencies
    data.f = data.count ./ data.nreads 

    # Get otus and runs for considered environment
    otus = unique(data.otu_id)
    runs = unique(data.run_id)
    otu_index = Dict(otu => i for (i, otu) in enumerate(otus))
    run_index = Dict(run => i for (i, run) in enumerate(runs))

    # Initialize matrix with zeros
    y = zeros(length(runs), length(otus))
    
    otu_groups = groupby(data, :otu_id)
    for g in otu_groups
        otu = g.otu_id[1]
        i = otu_index[otu]
        for (run, fval) in zip(g.run_id, g.f)
            j = run_index[run]
            y[j, i] = fval
        end
    end

    zero_counts = sum(y .== 0, dims=1)
    perm = sortperm(vec(zero_counts))

    # env_matrix["$key"] = y[:, perm]

    y = y[:, perm]
    s = size(y)
    
    all_vals = []
    for i in 1:size(y,1)
        print(i, "/$(size(y,1))\r")
        LRM = make_LRM(y[i,:], 0.4)
        vals = vec(LRM)
        push!(all_vals, vals[vals .> 0.0])
    end
    
    all_vals = vcat(all_vals...)
    
    bmin = round(minimum(all_vals))
    bmax = round(maximum(all_vals))
    Δb = (bmax - bmin) / 30
    fh = FHist.Hist1D(all_vals, binedges=bmin:Δb:bmax)
    
    μ, σ = mean(fh), std(fh)
    centers = bincenters(fh)
    centers .-= μ
    centers ./= σ
    norm_counts = bincounts(fh) ./ (integral(fh) * Δb)
    
    valid = norm_counts .> 0.0
    yy = log.(norm_counts[valid])
    centers = centers[valid]
    
    scatter!(fig, centers, yy, label="$key $s")

    Pkg.gc()

end

SEAWATER


      Active manifest files: 3 found
      Active 

RIVER


artifact files: 214 found
      Active scratchspaces: 2 found
     Deleted no artifacts, repos, packages or scratchspaces


      Active manifest files: 3 found
      Active artifact files: 214 found
      Active scratchspaces: 2 found
     Deleted 

...

no artifacts, repos, packages or scratchspaces


ORALCAVITY


      Active manifest files: 3 found
      Active artifact files: 214 found
      Active scratchspaces: 2 found
     Deleted no artifacts, repos, packages or scratchspaces
      Active manifest files: 3 found
      Active 

GUT


artifact files: 214 found
      Active scratchspaces: 2 found
     Deleted no artifacts, repos, packages or scratchspaces


In [ ]:
display(fig)